In [6]:
import os
import librosa
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the path and classes
base_path = "/content/drive/MyDrive/emotionDetection/AudioData"
classes = {'a': 'anger', 'd': 'disgust', 'f': 'fear', 'h': 'happiness', 'n': 'neutral', 'sa': 'sadness', 'su': 'surprise'}
actor_folders = ['DC', 'JE', 'JK', 'KL']

# Load the data
def load_data():
    labels = []
    features = []
    for actor in actor_folders:
        actor_folder = os.path.join(base_path, actor)
        for file_name in os.listdir(actor_folder):
            emotion_prefix = file_name.split('0')[0] if '0' in file_name else file_name[0]
            if len(emotion_prefix) > 1 and emotion_prefix[0] == 's':
                emotion_prefix = emotion_prefix[:2]
            else:
                emotion_prefix = emotion_prefix[0]
            emotion_label = classes.get(emotion_prefix, 'unknown')
            labels.append(emotion_label)
            file_path = os.path.join(actor_folder, file_name)
            features.append(extract_features(file_path))
    return np.array(features), np.array(labels)

# Extract features
def extract_features(file_path):
    audio_data, sample_rate = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
    return mfccs_processed

# Load the data
features, labels = load_data()

# Convert labels to integer
le = LabelEncoder()
labels = le.fit_transform(labels)

# Convert labels to categorical one-hot encoding
labels_categorical = to_categorical(labels)

# Split your data into a training set and a test set
features_train, features_test, labels_train, labels_test = train_test_split(features, labels_categorical, test_size=0.2)

# Reshape features for RNN input
features_train = np.reshape(features_train, (features_train.shape[0], 1, features_train.shape[1]))
features_test = np.reshape(features_test, (features_test.shape[0], 1, features_test.shape[1]))

# Define the model
model = Sequential()

# Add layers
model.add(LSTM(256, return_sequences=True, input_shape=(features_train.shape[1], features_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(labels_categorical.shape[1], activation='softmax'))  # Assuming you have 7 emotion classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(features_train, labels_train, epochs=100, batch_size=96, validation_data=(features_test, labels_test))

# Evaluate the model
score = model.evaluate(features_test, labels_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/100
4/4 [==============================] - 5s 243ms/step - loss: 2.0752 - accuracy: 0.1849 - val_loss: 2.0214 - val_accuracy: 0.1979
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 2.0035 - accuracy: 0.2708 - val_loss: 2.0159 - val_accuracy: 0.1979
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 1.9897 - accuracy: 0.2708 - val_loss: 1.9933 - val_accuracy: 0.1979
Epoch 4/100
4/4 [==============================] - 0s 14ms/step - loss: 1.9563 - accuracy: 0.2839 - val_loss: 1.9424 - val_accuracy: 0.2292
Epoch 5/100
4/4 [==============================] - 0s 12ms/step - loss: 1.8957 - accuracy: 0.3333 - val_loss: 1.8898 - val_accuracy: 0.2917
Epoch 6/100
4/4 [==============================] - 0s 12ms/step - loss: 1.8585 - accuracy: 0.3672 - val_loss: 1.8396 - val_accuracy: 0.3229
Epoch 7/100
4/4 [==============================] - 0s 12ms/step - loss: 1.7790 - accuracy: 0.4062 - val_loss: 1.7837 - val_accuracy: 0.3542
Epoch 8/100
4/4 [==